In [ ]:
# Maybe useful:
#     https://github.com/cridin1/malware-classification-CNN

In [2]:
import pandas as pd
import pefile
from PIL import Image
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from keras.regularizers import l2
from keras.layers import LeakyReLU

In [3]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image size and batch size
img_height, img_width = 128, 128  # Adjust based on your image size
batch_size = 32

# Define ImageDataGenerator for preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
)

# Load and preprocess the images from the directories
train_generator = train_datagen.flow_from_directory(
    'data',  # Parent directory containing '0' and '1'
    target_size=(img_height, img_width),  # Resize images to a consistent size
    batch_size=batch_size,
    class_mode='binary',  # Binary classification (Class A vs. Class B)
    subset='training',  # Split data for training
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    'data',  # Parent directory containing '0' and '1'
    target_size=(img_height, img_width),  # Resize images to a consistent size
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    subset='validation',  # Split data for validation
    shuffle=True
)


Found 171318 images belonging to 2 classes.
Found 30231 images belonging to 2 classes.


In [4]:
def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(128, 128, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Flatten(),
        
        Dense(units=256),
        LeakyReLU(alpha=0.2),  # Use LeakyReLU for more robust activation
        Dropout(0.5),
        
        Dense(units=128),  # Additional dense layer
        LeakyReLU(alpha=0.2),
        Dropout(0.5),
        
        Dense(units=1, activation='sigmoid')  # Final output layer
    ])
    
    return model


In [5]:
cnn_model = create_model()

c:\Users\Sam\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Sam\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [6]:
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 124, 124, 128)  │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 124, 124, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 60, 60, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 60, 60, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,605,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,741,793 (6.64 MB)

 Trainable params: 1,741,345 (6.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [7]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)
optimizer = Adam(learning_rate=0.001)
cnn_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['Recall'])

In [8]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assuming `train_generator.classes` contains all the labels in the training set
train_labels = train_generator.classes  # Get the labels for the training set


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.where(tf.equal(y_true, 0), 2.5, 1.0)  # Assign weights for goodware and malware
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) * weights

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  # Increased patience
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

cnn_model.compile(
    optimizer='adam',
    loss=weighted_binary_crossentropy,
    metrics=['Recall']
)

# Train the model without `class_weight`
history = cnn_model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)


c:\Users\Sam\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
5354/5354 ━━━━━━━━━━━━━━━━━━━━ 3722s 695ms/step - Recall: 0.8024 - loss: 1.0936 - val_Recall: 0.8885 - val_loss: 0.5862 - learning_rate: 0.0010
Epoch 2/20
5354/5354 ━━━━━━━━━━━━━━━━━━━━ 2208s 412ms/step - Recall: 0.8716 - loss: 0.5590 - val_Recall: 0.8310 - val_loss: 0.6960 - learning_rate: 0.0010
Epoch 3/20
5354/5354 ━━━━━━━━━━━━━━━━━━━━ 1882s 352ms/step - Recall: 0.8858 - loss: 0.5058 - val_Recall: 0.8093 - val_loss: 0.6073 - learning_rate: 0.0010
Epoch 4/20
5354/5354 ━━━━━━━━━━━━━━━━━━━━ 1890s 353ms/step - Recall: 0.8948 - loss: 0.4693 - val_Recall: 0.9713 - val_loss: 0.5153 - learning_rate: 0.0010
Epoch 5/20
5354/5354 ━━━━━━━━━━━━━━━━━━━━ 2241s 419ms/step - Recall: 0.8990 - loss: 0.4489 - val_Recall: 0.9332 - val_loss: 0.3510 - learning_rate: 0.0010
Epoch 6/20
5354/5354 ━━━━━━━━━━━━━━━━━━━━ 1882s 351ms/step - Recall: 0.9016 - loss: 0.4380 - val_Recall: 0.9631 - val_loss: 0.5337 - learning_rate: 0.0010
Epoch 7/20
5354/5354 ━━━━━━━━━━━━━━━━━━━━ 1883s 352ms/step - Recall: 0

In [10]:
# import tensorflow as tf
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# import numpy as np


# class_weights = {0: 1.5, 1: 1.0}  # Adjust weights; higher weight for goodware (label 0)


# cnn_model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['Recall']
# )


# # Callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  # Increased patience
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# # Train the model
# history = cnn_model.fit(
#     train_generator,
#     epochs=10,  # Increased from 10 to 30 for better learning
#     validation_data=validation_generator,
#     callbacks=[early_stopping, reduce_lr],
#     class_weight=class_weights
# )


In [11]:
# Evaluate the model on the validation data
validation_loss, validation_accuracy = cnn_model.evaluate(validation_generator)

945/945 ━━━━━━━━━━━━━━━━━━━━ 114s 121ms/step - Recall: 0.9533 - loss: 0.2259


In [12]:
# Predict using new data (use this for new images)
predictions = cnn_model.predict(validation_generator)

945/945 ━━━━━━━━━━━━━━━━━━━━ 112s 118ms/step


In [13]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

predicted_classes = np.round(predictions).astype(int)
true_classes = validation_generator.classes

print(classification_report(true_classes, predicted_classes, target_names=validation_generator.class_indices.keys()))

print(confusion_matrix(true_classes, predicted_classes))

              precision    recall  f1-score   support

           0       0.42      0.43      0.42     12771
           1       0.57      0.57      0.57     17460

    accuracy                           0.51     30231
   macro avg       0.50      0.50      0.50     30231
weighted avg       0.51      0.51      0.51     30231

[[5454 7317]
 [7562 9898]]


In [48]:
# Save the model in the native Keras format
model_save_path = "saved_model4.keras"
cnn_model.save(model_save_path, include_optimizer=False)
print(f"Model saved in native Keras format to {model_save_path}")


Model saved in native Keras format to saved_model4.keras


In [1]:
from tensorflow.keras.models import load_model

cnn_model = load_model("saved_model4.keras", compile=False)
cnn_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", "Recall"]
)


In [2]:
import os
import numpy as np
from PIL import Image

def bytes_to_rgb_image(byte_data, image_size=(128, 128)):
    """
    Convert raw byte data to a grayscale image resized to the given image size.

    Args:
        byte_data (bytes): The raw bytes of the file.
        image_size (tuple): Target size for the image (width, height).

    Returns:
        np.ndarray: An RGB image array suitable for model input.
    """
    # Ensure the byte data is a numpy array of integers
    byte_array = np.frombuffer(byte_data, dtype=np.uint8)

    # Reshape into a square; pad or trim if necessary
    side_length = int(np.ceil(np.sqrt(len(byte_array))))
    padded_array = np.zeros(side_length ** 2, dtype=np.uint8)
    padded_array[:len(byte_array)] = byte_array
    square_image = padded_array.reshape((side_length, side_length))

    # Convert to RGB format
    image = Image.fromarray(square_image, mode='L').convert('RGB')
    image = image.resize(image_size)

    # Normalize to range [0, 1] and add a batch dimension
    return np.expand_dims(np.array(image) / 255.0, axis=0)

def predict_folder_with_debug(model, folder_path, threshold=0.87):
    """
    Predicts malware/goodware in a folder with debugging information.
    Args:
        model: Trained model to use for prediction.
        folder_path (str): Path to the folder containing files to predict.
        threshold (float): Threshold for classification. Default is 0.5.
    """
    print("Model loaded successfully.")

    # Check if folder path exists
    if not os.path.exists(folder_path):
        print(f"Error: The folder path '{folder_path}' does not exist.")
        return

    # Check if the folder is empty
    if not os.listdir(folder_path):
        print(f"Error: The folder '{folder_path}' is empty.")
        return

    total_files = 0
    malware_count = 0
    goodware_count = 0

    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            total_files += 1

            try:
                # Read the file as bytes
                with open(file_path, "rb") as f:
                    byte_data = f.read()
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")
                continue

            try:
                # Convert bytes to an image
                processed_image = bytes_to_rgb_image(byte_data)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                continue

            try:
                # Predict using the model
                prediction = model.predict(processed_image, verbose=0)
                predicted_class = 1 if prediction[0][0] >= threshold else 0
                print(f"Prediction for {file_path}: {prediction[0][0]:.4f} (Threshold: {threshold})")
            except Exception as e:
                print(f"Error predicting file {file_path}: {e}")
                continue

            # Update counts
            if predicted_class == 1:
                malware_count += 1
            else:
                goodware_count += 1

    if total_files == 0:
        print("No files were found in the specified folder.")
        return

    # Print results
    print(f"Total files scanned: {total_files}")
    print(f"Malware: {malware_count} ({(malware_count / total_files) * 100:.2f}%)")
    print(f"Goodware: {goodware_count} ({(goodware_count / total_files) * 100:.2f}%)")

# Example Usage
# model = load_model("saved_model.keras", compile=False)
# folder_path = "datasets/datasets/gw1"
# predict_folder_with_debug(model, folder_path, threshold=0.55)


In [3]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/gw1")

Model loaded successfully.
Prediction for datasets/datasets/gw1\0000: 1.0000 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0001: 0.9890 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0002: 0.9885 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0003: 1.0000 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0004: 0.0606 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0005: 0.9861 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0006: 0.9761 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0007: 0.0591 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0008: 0.9997 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0009: 0.9895 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0010: 0.9260 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0011: 0.4152 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/gw1\0013: 0.9891 (Threshold: 0.87)
Prediction for datase

In [4]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/gw2")

Model loaded successfully.
Prediction for datasets/datasets/gw2\0000: 0.9941 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0001: 0.9961 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0002: 0.9848 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0003: 0.9604 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0004: 0.9615 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0005: 0.9986 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0006: 0.9999 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0007: 0.9198 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0008: 0.9979 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0009: 0.9828 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0010: 0.9874 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0011: 0.9857 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0012: 0.9716 (Threshold: 0.87)
Prediction for datasets/datasets/gw2\0013: 0.9734 (Threshold: 0.87)
Prediction for datase

In [5]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/gw3")

Model loaded successfully.
Prediction for datasets/datasets/gw3\0000: 0.9712 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0001: 0.9698 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0002: 0.9424 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0003: 0.9925 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0004: 0.0446 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0005: 0.9718 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0006: 0.9770 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0007: 0.9658 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0008: 0.9940 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0009: 0.9800 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0010: 0.9926 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0011: 0.9845 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0012: 0.9663 (Threshold: 0.87)
Prediction for datasets/datasets/gw3\0013: 0.9653 (Threshold: 0.87)
Prediction for datase

In [6]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/gw4")

Model loaded successfully.
Prediction for datasets/datasets/gw4\0000: 0.9994 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0001: 0.9991 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0002: 0.0390 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0003: 0.0484 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0004: 0.8563 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0005: 0.5985 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0006: 0.1965 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0007: 0.9998 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0008: 0.9661 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0009: 0.9443 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0010: 0.9991 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0011: 0.9812 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0012: 0.9994 (Threshold: 0.87)
Prediction for datasets/datasets/gw4\0013: 0.0747 (Threshold: 0.87)
Prediction for datase

In [7]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/gw5")

Model loaded successfully.
Prediction for datasets/datasets/gw5\0000: 1.0000 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0001: 0.8896 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0002: 0.9997 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0003: 0.9830 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0004: 0.9755 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0005: 0.9457 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0006: 0.9168 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0007: 0.7470 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0008: 0.9753 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0009: 0.9791 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0010: 0.7785 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0011: 0.9915 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0012: 0.9845 (Threshold: 0.87)
Prediction for datasets/datasets/gw5\0013: 0.9818 (Threshold: 0.87)
Prediction for datase

In [8]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/gw6")

Model loaded successfully.
Prediction for datasets/datasets/gw6\0000: 0.1683 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0001: 0.6717 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0002: 0.0090 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0003: 0.0027 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0004: 0.0189 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0005: 0.0017 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0006: 0.0159 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0007: 0.8361 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0008: 0.7303 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0009: 0.0176 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0010: 0.1705 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0011: 0.0074 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0012: 0.1141 (Threshold: 0.87)
Prediction for datasets/datasets/gw6\0013: 0.0174 (Threshold: 0.87)
Prediction for datase

In [9]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw1")

Model loaded successfully.
Prediction for datasets/datasets/mw1\0000: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0001: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0002: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0003: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0004: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0005: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0008: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0009: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0010: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0011: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw1\0014: 0.9891 (Threshold: 0.87)
Prediction for datase

In [10]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw2")

Model loaded successfully.
Prediction for datasets/datasets/mw2\0000: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0001: 0.4992 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0002: 0.3022 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0003: 0.4613 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0004: 1.0000 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0005: 0.9993 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0008: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0009: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0010: 1.0000 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0011: 0.9999 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw2\0013: 0.9999 (Threshold: 0.87)
Prediction for datase

In [11]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw3")

Model loaded successfully.
Prediction for datasets/datasets/mw3\0000: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0001: 0.8187 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0002: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0003: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0004: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0005: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0008: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0009: 0.5263 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0010: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0011: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw3\0013: 0.9891 (Threshold: 0.87)
Prediction for datase

In [12]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw4")

Model loaded successfully.
Prediction for datasets/datasets/mw4\001: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\002: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\003: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\004: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\005: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\008: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\009: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\010: 0.9986 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\011: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\013: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw4\014: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw

In [13]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw5")

Model loaded successfully.
Prediction for datasets/datasets/mw5\001: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\002: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\003: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\004: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\005: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\008: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\009: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\010: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\011: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\013: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw5\014: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw

In [14]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw6")

Model loaded successfully.
Prediction for datasets/datasets/mw6\001: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\002: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\003: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\004: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\005: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\008: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\009: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\010: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\011: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\013: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw6\014: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw

In [15]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw7")

Model loaded successfully.
Prediction for datasets/datasets/mw7\0000: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0001: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0002: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0003: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0004: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0005: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0008: 0.9999 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0009: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0010: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0011: 0.9066 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw7\0013: 0.9891 (Threshold: 0.87)
Prediction for datase

In [16]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw8")

Model loaded successfully.
Prediction for datasets/datasets/mw8\0000: 0.3895 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0001: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0002: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0003: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0004: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0005: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0006: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0007: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0008: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0009: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0010: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0011: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0012: 0.9891 (Threshold: 0.87)
Prediction for datasets/datasets/mw8\0013: 0.9891 (Threshold: 0.87)
Prediction for datase

In [60]:
model = load_model("saved_model4.keras", compile=False)
predict_folder_with_debug(model, "datasets/datasets/mw9")

Model loaded successfully.
Error: The folder 'datasets/datasets/mw9' is empty.
